In [ ]:
import os
import math

In [ ]:
def read_k_data(file_path):
    """ Read kt vs time data from Lammps simulation output file """
    dt = 5
    with open(file_path, 'r') as j:
        a = []
        time = []
        for line_index, line in enumerate(j, start=1):
            if line_index >= 200015: 
                l = line.split()
                t = (float(l[0]) - 1) * dt / 1000.0
                a.append(float(l[3]))
                time.append(t)
    return a, time

def convert_k_data(k_data):
    """ Convert Lammps output data to W/mK thermal conductivity """
    volume = 80 * 80 * 80 
    dt = 5
    k_b = 0.001987
    T = 300.0
    conversion = 69443.84
    new_kt = []
    for data_index, data in enumerate(k_data):
        if data_index == 0:
            kt =  data / 2 * volume * dt / (k_b * math.pow(T, 2)) * conversion
            new_kt.append(kt)
        if data_index > 0:
            kt =  kt + data * volume * dt / (k_b * math.pow(T, 2)) * conversion
            new_kt.append(kt)
    return new_kt

def avg_k_data(k_data_list):
    """ Calculate average of thermal conductivity for multiple runs """
    n = len(k_data_list[0])
    for i in k_data_list:
        m = len(i)
        if m != n:
            print('Data mismatch', i)
    avg_data = []
    for data_index in range(n):
        data = sum([i[data_index] for i in k_data_list])/len(k_data_list)
        avg_data.append(data)
    return avg_data

def get_kt(kt, time, t0=5, t1=10):
    """ Get approximate thermal conductivity value for a single run """
    start = time.index(t0)
    end = time.index(t1)
    kt_data = []
    for i in range(start, end):
        kt_data.append(kt[i])
    kt_avg = sum(kt[start:end]) / len(kt[start:end])
    return kt_avg

def read_runs(trial_dir):
    """ Read multiple runs for single trial """
    run_data = []
    print('\n------ %s ------' % os.path.split(trial_dir)[-1])
    for run_index, run in enumerate(os.listdir(trial_dir)):
        run_dir = os.path.join(trial_dir, run)
        if os.path.isdir(run_dir):
            data_path = os.path.join(run_dir, 'J0Jt_t.dat')
            kt, time = read_k_data(data_path)
            k_data = convert_k_data(kt)
            run_data.append(k_data)
            print('%s -> kt: %.3f W/mK' % (run, get_kt(k_data, time)))
    avg_kt = avg_k_data(run_data)
    print('Average -> %.3f W/mK' % get_kt(avg_kt, time))
    return run_data, avg_kt, time

def read_trials(mult_trial_dir):
    """ Read multiple trials with multiple runs"""
    trial_data = []
    for single_trial in os.listdir(mult_trial_dir):
        single_trial_dir = os.path.join(mult_trial_dir, single_trial)
        if os.path.isdir(single_trial_dir):
            run_data, avg_kt, time = read_runs(single_trial_dir)
            trial_data.append(avg_kt)
    return trial_data

## Read results for single trial and 5 runs

In [ ]:
trial_dir = os.path.join(os.getcwd(), 'Results', 'Trial4', 'Trial3-1')

run_data, avg_kt, time = read_runs(trial_dir)

## Plot results for each run and average 

In [ ]:
%pylab inline
import matplotlib.pyplot as plt

limit = 2000
lgnd = []
for i, rd in enumerate(run_data, start=1):
    plt.plot(time[:limit], rd[:limit])
    lgnd.append('Run%i' % i)

plt.plot(time[:limit], avg_kt[:limit], '--k')
lgnd.append('Average')

plt.title('Single MOF Run')
plt.ylabel('kt')
plt.xlabel('Time')

plt.legend(lgnd, loc=4, fontsize=10)

## Read multiple trials with multiple runs

In [ ]:
mult_trial_dir = os.path.join(os.getcwd(), 'Results', 'Trial4')

trial_data = read_trials(mult_trial_dir)

## Plot multiple trials

In [ ]:
%pylab inline
import matplotlib.pyplot as plt

trial_id = [1, 4, 6, 3, 5, 2]
limit = 2000
lgnd = []
for i, rd in enumerate(trial_data, start=0):
    plt.plot(time[:limit], rd[:limit])
    lgnd.append('Trial%i' % trial_id[i])

plt.title('Multiple Trials')
plt.ylabel('kt')
plt.xlabel('Time')

plt.legend(lgnd, loc=4, fontsize=10)